In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

from zipline import run_algorithm
from zipline.api import (
    attach_pipeline,
    date_rules,
    order_target_percent,
    pipeline_output,
    record,
    symbols,
    schedule_function,
    symbol,
    time_rules,
    get_open_orders,
)
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import SimpleMovingAverage
from zipline.pipeline.data import USEquityPricing

import pyfolio as pf

In [2]:
import os

from zipline.utils.run_algo import load_extensions

load_extensions(
    default=True,
    extensions=[],
    strict=True,
    environ=os.environ,
)

In [3]:
import os
os.chdir('C:\\Users\\avi\\.zipline\\csv\\yahoo')

In [4]:
# import yfinance as yf
# for symbol in ["SPY", "EFA", "IEF", "VNQ", "GSG","AAPL","MSFT"]:
# # for symbol in ["MSFT"]:
#     data = yf.download(symbol, multi_level_index=False)
#     data = data.loc[data.index > '1999-01-01']
#     data.to_csv('%s.csv'%symbol)
    

In [ ]:
# set YAHOO_CSVDIR=/path/to/csvdir 
# zipline ingest -b yahoo_csv

In [5]:
def initialize(context):
    context.stocks = [
#     context.symbols = [
        symbol("SPY"),
        symbol("EFA"),
        symbol("IEF"),
        symbol("VNQ"),
        symbol("GSG"),
    ]
#     context.stocks = symbols("SPY", "EFA", "IEF", "VNQ", "GSG")    
    #SPY - S&P 500 ETF Trust
    #EFA - MSCI EAFE ETF
    #IEF - 10 Year Treasury Bond ETF
    #VNQ - Vanguard Real Estate Index Fund ETF
    #GSG - S&P GSCI Commodity-Indexed Trust ETF    
    
    context.sma = {}
    context.period = 10 * 21

    #for asset in context.symbols: 
    for asset in context.stocks: 
        context.sma[asset] = SimpleMovingAverage(
            inputs=[USEquityPricing.close],
            window_length=context.period
        )
    
    
    #The rebalance method will be called for every single month_start with the prices 1
    #min after the market opens
    schedule_function(
        func=rebalance,
        date_rule=date_rules.month_start(),
        time_rule=time_rules.market_open(minutes=1),
    )

    context.set_commission(
        commission.PerShare(cost=0.01, min_trade_cost=1.00)
    )
    context.set_slippage(slippage.VolumeShareSlippage())

The logic is straightforward: For each ETF representing an asset class, calculate the 10-
month simple moving average using the closing price.
This is a long-term strategy that rebalances at the market open on the first day of the
month. Zipline’s schedule function makes it simple to set this up. Note we schedule a
function called rebalance to run on this schedule. 

In [6]:
def rebalance(context, data):
    
    longs = [
        asset
        for asset in context.stocks
        if data.current(asset, "price") > context.sma[asset].mean()
    ]

    for asset in context.portfolio.positions:
        if asset not in longs and data.can_trade(asset):
            order_target_percent(asset, 0)

    for asset in longs:
        if data.can_trade(asset):
            order_target_percent(asset, 1.0 / len(longs))

On the first trading day of every month, the logic checks if the current price is greater than
the 10-month simple moving average. If so, it adds the symbol to a list.
The next step sets a target of 0% for those assets with a price that does not exceed the 10-
month simple moving average.
Finally, we equal-weight the portfolio with the ETFs that are trending

In [76]:
######### The below code is just a test to check yahoo csv ingest ##################### 
# %matplotlib inline

# from zipline import run_algorithm 
# from zipline.api import order_target_percent, symbol, set_commission
# from zipline.finance.commission import PerTrade
# import pandas as pd
# import pyfolio as pf

# import warnings
# warnings.filterwarnings('ignore')

# def initialize(context): 
#     context.stock = symbol('AAPL')
#     context.rolling_window = 90
#     set_commission(PerTrade(cost=5))     

# def handle_data(context, data): 
#     price_hist = data.history(context.stock, "close", context.rolling_window, "1d")          
    
#     rolling_mean_short_term = price_hist.ewm(span=5,adjust=True,ignore_na=True).mean()
#     rolling_mean_long_term = price_hist.ewm(span=30,adjust=True,ignore_na=True).mean()
    
#     if rolling_mean_short_term[-1] > rolling_mean_long_term[-1]:
#         order_target_percent(context.stock, 1.0)     
#     elif rolling_mean_short_term[-1] < rolling_mean_long_term[-1]:
#         order_target_percent(context.stock, 0.0)     

# def analyze(context, perf): 
#     returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf) 
#     pf.create_returns_tear_sheet(returns, benchmark_rets = None)
    
# # start_date = pd.to_datetime('2000-1-1', utc=True)
# # end_date = pd.to_datetime('2018-1-1', utc=True)
# start_date = pd.Timestamp('2000-1-1')
# end_date = pd.Timestamp('2018-1-1')
    
# results = run_algorithm(start = start_date, end = end_date, initialize = initialize, 
#                         analyze = analyze, handle_data = handle_data, capital_base = 10000, 
#                         data_frequency = 'daily', 
#                         #bundle ='quandl'
#                         bundle ='yahoo_csv'
#                        )

In [7]:
start = pd.Timestamp("2000")
end = pd.Timestamp("2015-12-30")

perf = run_algorithm(
    start=start,
    end=end,
    initialize=initialize,
    capital_base=100000,
#     bundle="quantopian-quandl"
#     bundle="quandl",
#     bundle="yahoo_direct"
    bundle = "yahoo_csv"
)

ValueError: 'nan' is not a valid variable name

In [67]:
returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)


pf.create_full_tear_sheet(
    returns,
    positions=positions,
    transactions=transactions,
    round_trips=True,
)

AttributeError: 'DataFrame' object has no attribute 'amount'